## Setup

DEPENDENCIES

In [ ]:
!pip install -q together openai groq regex

In [ ]:
import os
import json
import re
import regex
from together import Together
from openai import OpenAI
from groq import Groq
from typing import List, Dict, Any

API KEYs & CLIENTS

In [ ]:
os.environ["TOGETHER_API_KEY"] = "PUT YOUR API KEY HERE"
os.environ["GROQ_API_KEY"] = "PUT YOUR API KEY HERE"
os.environ["OPENAI_API_KEY"] = "PUT YOUR API KEY HERE"

In [ ]:
tg_client = Together(api_key=os.environ.get("TOGETHER_API_KEY"))
gq_client = Groq(api_key=os.environ.get("GROQ_API_KEY"))
op_client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

CALL LLM

In [ ]:
provider="together"

def call_llm (question : str):
  if provider == "together":
    response = tg_client.chat.completions.create(
      model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
      messages=[
          {
              "role": "user",
              "content": question
          }
      ]
    )
    return response.choices[0].message.content
  elif provider == "groq":
    response = gq_client.chat.completions.create(
        model="groq/compound-mini",
        messages=[
            {
              "role": "user",
              "content":question
            }
        ]
    )
    return(response.choices[0].message.content)
  else:
    response = op_client.responses.create(
        model="gpt-5",
        input=question,
        reasoning={ "effort": "low" },
        text={ "verbosity": "low" },
    )

    return(response.output_text)

OUTPUT FORMATTING HELPERS

In [ ]:
def cleanCODE(text):
    # Split the text into lines
    lines = text.strip().split('\n')

    if lines and lines[0].startswith('```'):
        lines = lines[1:]

    closing_index = None
    for i, line in enumerate(lines):
        if '```' in line:
            closing_index = i
            break

    if closing_index is not None:
        lines = lines[:closing_index]
    clean_text = '\n'.join(lines)

    return clean_text

def cleanJSON(text):
    matchy = regex.search(r'\{(?:[^{}]|(?R))*\}', text)
    return(matchy.group())


def safe_json_parse(response: str, fallback: Any = None) -> Any:
    try:
        clean_response = cleanJSON(response)
        return json.loads(clean_response)
    except json.JSONDecodeError:
        # Fix invalid escape sequences by double-escaping
        clean_response = re.sub(r'\\(?!["\\/bfnrtu])', r'\\\\', clean_response)
        try:
            return json.loads(clean_response)
        except json.JSONDecodeError:
            return fallback

## MAIN AGENT FUNCTIONS

TASK ANALYSIS

In [ ]:
def analyze_problem(code: str) -> Dict[str, Any]:
    prompt =f"""
    Role: You are the Main Analysis Agent. Your responsibility
    is to perform a systematic static analysis of the given
    code and identify defects that prevent correct execution.

    Input: Buggy code: {code}

    Instructions:
    1. Assess the complexity of the debugging task and classify
    it as SIMPLE or COMPLEX based on the following criterias:
      - Number of critical bugs
      - Degree of bug isolation
      - Clarity of control flow
      - Concurrency or resource management issues
      - Inter-function or module coupling
    2. Identify and precisely locate all bugs that may prevent
    the code from executing correctly. For each bug, specify:
      - Bug type (e.g., syntax error, API misuse, ...)
      - Location (function name, line number range)
      - Brief explanation of why it causes failure
    3. Produce a structured, step-by-step repair plan
    describing how the identified bugs should be fixed.

    Output:Return a JSON object with the following schema:
    {{
      complexity": "SIMPLE" or "COMPLEX",
      "bugs": [
        {{
          "type": "...",
          "location": "...",
          "explanation": "..."
        }}
      ],
      "plan": "..."
    }}
    Do not write any text besides the JSON.
    """
    result = call_llm(prompt)
    parsed = safe_json_parse(result, fallback={"complexity": "SIMPLE", "bugs":"...", "plan": "..."})
    return parsed

NEW ITERATION ANALYSIS

In [ ]:
def new_iteration_analyze_problem(code: str, failure_log: Dict[str, Any], previous_plan: Dict[str, Any]) -> Dict[str, Any]:
    prompt =f"""
    Role:
        You are the Main Analysis Agent. Your responsibility is to perform a systematic review
        of the given code after a previous fix attempt has failed. Identify all remaining defects
        that prevent execution and generate a precise new repair plan. Do not provide suggestions
        or potential improvements.

    Input:
        Previous repair plan: {previous_plan}
        Failure summary: {failure_log["summary"]}
        Buggy code: {code}

    Instructions:
    1. Assess the complexity of the remaining debugging task and classify it as SIMPLE or COMPLEX
      using the following criteria:
          - Number of critical bugs remaining
          - Degree of bug isolation
          - Clarity of control flow
          - Presence of concurrency or resource management issues
          - Dependencies between functions or modules
    2. Identify all remaining bugs, specifying for each:
          - Bug type (e.g., syntax error, API misuse, logic error)
          - Location (function name, line number range)
          - Explanation of why it prevents correct execution
    3. Produce a step-by-step new repair plan detailing how to fix the remaining bugs.

    Output:
    Return a JSON object with this schema:
    {{
      "complexity": "SIMPLE" or "COMPLEX",
      "bugs": [
        {{
          "type": "...",
          "location": "...",
          "explanation": "..."
        }}
      ],
      "plan":"..."
    }}
    Do not write any text besides the JSON.
    """
    result = call_llm(prompt)
    parsed = safe_json_parse(result, fallback={"complexity": "SIMPLE", "bugs":"...", "plan": "..."})
    return parsed

AGENTS CREATION AND PRIORITIZATION

In [ ]:
def generate_agents(bugs: List[Any], plan: str) -> List[Dict[str, Any]]:
    prompt = f"""
    Role:
        You are the Main Agent responsible for creating and organizing specialized agent profiles.
        Your task is twofold:
        1. Generate the minimum set of agents required to fix the located bugs.
        2. Prioritize these agents based on their dependencies to ensure correct execution order.

    Input:
        Located bugs: {bugs}
        Repair instructions: {plan}

    Instructions:

    Step 1 - Agent Generation:
        1. Generate the minimum set of agents required to fix the bugs.
        2. For each agent, provide:
            - name
            - role
            - task_description (phrased as "Your task is to..." and explicitly referencing the located errors)

    Step 2 - Agent Prioritization:
        1. Determine dependencies between agents (e.g., syntax must be fixed before logic errors).
        2. Order the agents based on these dependencies.

    Output:
    Return a JSON object with the following schema:

    {{
      "agents": [
        {{
          "name": "...",
          "role": "...",
          "task_description": "..."
        }}
      ],
      "execution_order": ["Agent_1_name", "Agent_2_name", "..."]
    }}

    Do not write any text besides the JSON.
    """

    result = call_llm(prompt)
    parsed = safe_json_parse(result, fallback={"agents": [], "execution_order": []})
    return parsed


ITERATIVE TASK REVIEW

In [ ]:
def task_review(agent: Dict[str, Any], agent_report: Dict[str, Any]) -> Dict[str, Any]:
    prompt = f"""
    Role:
    You are the Main Agent responsible of reviewing the output of a specialized debugging agent.
    Your task is to make the decision whether to approve or refine the agent fixes.

    Input:
    The agent task description,
    {agent["task_description"]}
    The proposed fixed code
    {agent_report["fixed_code"]}
    The fix explanation:
    {agent_report["fix_explanation"]}

    Instructions:
    1. Check whether the agent\'s changes are correct and complete.
    2. If yes, approve and move to the next agent.
    3. If no, instruct the same agent to refine its work.

    Your output in JSON format in the following fromat:
    {{
      "decision": "APPROVE" or "REFINE",
      "feedback": "..." // if refinement is needed
    }}

    Do not write any text besides the JSON.
    """
    result = call_llm(prompt)
    parsed = safe_json_parse(result, fallback={"decision": "", "feedback":""})
    return parsed

FINAL VALIDATION

In [ ]:
def validate_solution(code: str) -> Dict[str, Any]:
    prompt = f"""
    Role:
        You are the Master Agent responsible for final validation and closure.
        Your task is to verify whether the given code is fully fixed and executable.
        Focus only on bugs that prevent correct execution.

    Input:
        Buggy code: {code}

    Instructions:
    1. Check whether the code is free of execution-blocking bugs.
    2. If the code is fully fixed, confirm completion.
    3. If not, summarize and explain all remaining bugs.

    Output:
    Return a JSON object with this schema:

    {{
      "status": "FIXED" or "NOT FIXED",
      "summary": "...",
      "remaining_bugs": [
        {{
          "type": "...",
          "location": "...",
          "explanation": "..."
        }}
      ],
      "explanation": "..."
    }}

    Do not write any text besides the JSON.
    """

    result = call_llm(prompt)
    parsed = safe_json_parse(result, fallback={"status": "NOT FIXED", "summary": "Parsing failed.", "remaining_bugs": [], "explanation":""})
    return parsed

SIMPLE BUGS FAST FIX

In [ ]:
def simple_fix(bugs: List[Any], code: str) -> Dict[str, Any]:
    prompt = f"""
    Role:
        You are a Code Repair Expert. Your task is to fix all identified bugs in the given code
        and produce a fully corrected version.

    Input:
        Buggy code: {code}
        Summary of issues: {bugs}

    Instructions:
    1. Fix the code to eliminate all execution-blocking bugs.
    2. Provide a clear explanation of each fix applied.

    Output:
    Return a JSON object with all newlines and quotes escaped (e.g., \\n, \\"):

    {{
      "fixed_code": "...",
      "fix_explanation": "..."
    }}

    Do not write any text besides the JSON.
    """

    result = call_llm(prompt)
    parsed = safe_json_parse(result, fallback={"fixed_code": "", "fix_explanation":""})
    return parsed

## SPECIALIZED AGENT TASK EXECUTION



EXECUTE TASK

In [ ]:

def execute_agent(agent: Dict[str, Any], code: str) -> Dict[str, Any]:
    prompt = f"""
    Role:
        You are a {agent["role"]}.
        Your task is to {agent["task_description"]}.

    Input:
        Buggy code: {code}

    Instructions:
    1. Fix the code to address the issues within your responsibility.
    2. Provide a clear explanation of the fix applied.

    Output:
    Return a JSON object with all newlines and quotes escaped (e.g., \\n, \\"):

    {{
      "fixed_code": "...",
      "fix_explanation": "..."
    }}

    Do not write any text besides the JSON.
    """

    result = call_llm(prompt)
    parsed = safe_json_parse(result, fallback={"fixed_code": "", "fix_explanation":""})
    return parsed

REPEAT TASK

In [ ]:

def retry_execute_agent(agent: Dict[str, Any], code: str, feedback: str) -> Dict[str, Any]:
    prompt = f"""
    Role:
        You are a {agent["role"]}.
        Your task is to {agent["task_description"]}.
        You previously attempted a fix that failed. Now, fix the code again using the provided feedback.

    Input:
        Buggy code: {code}
        Feedback: {feedback}

    Instructions:
    1. Fix the code, taking the feedback into account.
    2. Provide a clear explanation of the fix applied.

    Output:
    Return a JSON object with all newlines and quotes escaped (e.g., \\n, \\"):

    {{
      "fixed_code": "...",
      "fix_explanation": "..."
    }}

    Do not write any text besides the JSON.
    """

    result = call_llm(prompt)
    parsed = safe_json_parse(result, fallback={"fixed_code": "", "fix_explanation":""})
    return parsed

## MAIN EXECUTION PIPELINE

In [ ]:
def adaptive_debugger(buggy_code: str, max_iterations: int = 5):
    analysis_report = analyze_problem(buggy_code)
    print(analysis_report)
    if not analysis_report["bugs"]:
        print("\n🟢 No errors found. Code is already fixed")
        return buggy_code

    for iteration in range(max_iterations):

        print(f"\n--------------ITERATION {iteration + 1}")

        if analysis_report["complexity"] == "SIMPLE":
            print("\n--------------USING SIMPLE FIX")
            code_to_debug=simple_fix(analysis_report["bugs"], buggy_code)

        else:
            print("\n--------------USING MULTI AGENTS")

            created_agents= generate_agents(analysis_report['bugs'], analysis_report['plan'])
            agent_profiles= created_agents["agents"]
            execution_order = created_agents["execution_order"]


            print("\nNumber of Created Agents:", len(agent_profiles))
            print("\nExecution Order: " +" -> ".join(execution_order))
            reports = []
            code_to_debug = buggy_code
            for agent_name in execution_order:
                print(f"\nExecuting Agent: {agent_name}")
                agent = next((a for a in agent_profiles if a["name"] == agent_name), None)
                if not agent:
                    continue
                decision=""
                feedback=""
                review_iterations=0
                while not(decision == "APPROVE") and review_iterations < 3:

                    if decision=="REFINE":
                        print(f"\nAgent: {agent_name} needs refinement. Retrying")
                        result = retry_execute_agent(agent, code_to_debug, feedback)
                    else:
                        result = execute_agent(agent, code_to_debug)

                    review = task_review(agent, result)
                    decision = review["decision"]
                    feedback = review["feedback"]

                    if decision=='APPROVE':
                        print(f"\nAgent: {agent_name} approved")
                        print(f"\nAgent: {agent_name} execution completed")

                    code_to_debug = result["fixed_code"]


                    review_iterations+=1


        validation = validate_solution(code_to_debug)
        status = validation["status"]
        summary = validation["summary"]
        re_bugs = validation["remaining_bugs"]
        if status == "FIXED":
            print(f"\n🟢 Bugs fixed successfully. Total number of iterations: {iteration + 1}\n")
            return code_to_debug

        print("\n🟡 Failed to fix the bugs. Retrying....")
        print(f"\nValidation report : \n {summary} \n {re_bugs}")
        prev_analysis_report = analysis_report
        analysis_report = new_iteration_analyze_problem(code_to_debug, validation, analysis_report)

    print("\n🔴 Failed to fix the bugs after max iterations.")
    return code_to_debug


## BUGGY CODE EXAMPLE


In [ ]:
example_buggy_code="""
def process_student_records(students, output_file):

    averages = {}
    for student in students:
        grades = student["grades"]
        avg = sum(grades) / (len(grades) - 1)
        averages[student["name"]] = avg

    top_student = max(averages, key=lambda k: averages[k])

    normalized = []
    for g in grades:
        normalized.append(g / max(grades))

    with open(output_file, "w") as f:
        f.write("Student Averages:\n")
        for name, avg in averages.items():
            f.write(f"{name}: {avg}\n")

        f.write(f"\nTop student: {top_student} with {averages[top_student]}\n")

    summary = "Processed {count} students".format(cnt=len(students))
    f.write(summary)

    f.close()

    import jsonn
    result = json.dumps({
        "averages": averages,
        "top_student": top_student
    })

    return result
"""

## EXECUTION EXAMPLE

In [ ]:
fixed_code = adaptive_debugger(example_buggy_code)
print("Final Output:\n", fixed_code)